**IMPORTS**

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import os
import csv
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,  precision_score, recall_score
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import shannon_entropy

In [ ]:
image = None
texture = None
species = None
fractal_dim = None
entropy = None
vein_density = None
color = None
file_path = 'features_data - Sheet1.csv'

**FUNCTIONS FOR PROCESSING IMAGE**

In [ ]:
def show(img_path):
  plt.imshow(img_path)
  plt.axis('off')
  plt.show()

def resizeImage(image):
  try:
    # Get the dimensions of the original image
    original_height, original_width, _ = image.shape

    # Specify the desired dimensions of the resized image
    desired_width = 300
    desired_height = 400

    # Calculate the aspect ratio of the original image
    aspect_ratio = original_width / original_height

    # Apply the aspect ratio to the desired dimensions
    if desired_width / aspect_ratio <= desired_height:
        # Use the desired width to calculate the height
        new_width = desired_width
        new_height = int(new_width / aspect_ratio)
    else:
        # Use the desired height to calculate the width
        new_height = desired_height
        new_width = int(new_height * aspect_ratio)

    # Resize the image
    resized_image = cv2.resize(image, (new_width, new_height))
        # Get dimensions of image
    height, width, channels = resized_image.shape

    # Calculate coordinates of middle pixel
    x = width // 2
    y = height // 2
    global color
    # Get color of middle pixel
    color = resized_image[y, x]
    color  = int(0.299*color[2] + 0.587*color[1] + 0.114*color[0])
    return resized_image
  except Exception as e:
    print("Error in resizing image", e)
    return image
def getDiameter(contour, circularity, rectangularity, ellipticity,area ):
  try:
    # Fit an ellipse to the largest contour
    if ellipticity > circularity and rectangularity:
      ellipse = cv2.fitEllipse(contour)

    # Extract the major and minor axes of the ellipse
      major_axis = max(ellipse[1])
      minor_axis = min(ellipse[1])

    # Calculate the diameter as the maximum of major and minor axes
      diameter = max(major_axis, minor_axis)
      
    elif circularity > ellipticity and rectangularity:
      diameter = 2 * np.sqrt((area * circularity**2) / np.pi)
    else:
      diameter = np.sqrt(area / rectangularity)
    return diameter
  except Exception as e:
    print("Error in getting the diameter", e)
    return 0  
def entropy(image):
    """Calculate the entropy of an image."""
    hist, _ = np.histogram(image, bins=np.arange(0, 257))
    hist = hist / np.sum(hist)
    hist = hist[hist > 0]
    return -np.sum(hist * np.log2(hist))
def getTexture(path):
  try:
    global fractal_dim
    global entropy
    # Read in leaf image
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

    # Compute grey-level co-occurrence matrix (GLCM)
    glcm = graycomatrix(img, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], symmetric=True, normed=True)

    # Compute contrast using GLCM
    contrast = graycoprops(glcm, 'contrast')

    # Compute entropy using GLCM
    entropy_val = shannon_entropy(glcm)

    # Compute fractal dimension using box counting method
    pixels = np.sum(img <= 127)
    scales = np.logspace(0, np.log2(min(img.shape))-1, num=20, base=2, dtype=int)
    ns = []
    for scale in scales:
        slices = [slice(0, s, scale) for s in img.shape]
        ns.append(np.sum(img[tuple(slices)] <= 127))  # fixed line
    coeffs = np.polyfit(np.log(scales), np.log(ns), 1)
    fractal_dim = -coeffs[0]


    fractal_dim = -coeffs[0]
    entropy = entropy_val
  except Exception as e:
    print("Error in getting Texture of the image", e)
    return None
def applyFilters(image):
  global vein_density
  try:
    # Apply Gaussian blur to reduce noise
    ksize = (3, 3)  # kernel size
    sigmaX = 3  # standard deviation in X direction
    blur = cv2.GaussianBlur(image, ksize, sigmaX)
    gray = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    # Apply vein detection algorithm
    dst = cv2.ximgproc.thinning(thresh, thinningType=cv2.ximgproc.THINNING_GUOHALL)

    # Calculate vein density
    num_veins = np.count_nonzero(dst)
    leaf_area = np.count_nonzero(thresh)
    vein_density = num_veins / leaf_area

    # Apply Median blur to reduce noise
    ksize = 5  # kernel size (must be odd)
    median = cv2.medianBlur(blur, ksize)

    # Apply Bilateral filter to reduce noise
    d = 1  # diameter of each pixel neighborhood
    sigmaColor = 100  # filter sigma in the color space
    sigmaSpace = 100  # filter sigma in the coordinate space
    bilateral = cv2.bilateralFilter(median, d, sigmaColor, sigmaSpace)

    return bilateral
  except Exception as e:
    print("Error in applying Filters image", e)
    return image
def flatten(filtered_image):
  try:
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(filtered_image, cv2.COLOR_BGR2GRAY)
    # Perform adaptive histogram equalization to normalize the image
    # You can adjust the clipLimit and tileGridSize parameters to control the normalization effect
    clahe = cv2.createCLAHE(clipLimit=0.5, tileGridSize=(8, 8))
    normalized_image = clahe.apply(gray_image)

    # Convert the normalized image back to BGR color space
    normalized_image = cv2.cvtColor(normalized_image, cv2.COLOR_GRAY2BGR)
    # Resize the mask to match the size of the input image
    mask = cv2.threshold(gray_image, 25, 255, cv2.THRESH_BINARY)[1]
    mask = cv2.resize(mask, (filtered_image.shape[1], filtered_image.shape[0]), interpolation=cv2.INTER_NEAREST)

    # Perform bitwise_and operation on the image and mask
    output_image = cv2.bitwise_and(filtered_image, filtered_image, mask=mask)

    # Perform bitwise_not operation on the mask
    not_mask = cv2.bitwise_not(mask)
    not_mask = cv2.resize(not_mask, (normalized_image.shape[1], normalized_image.shape[0]), interpolation=cv2.INTER_NEAREST)

    normalized_not_mask = cv2.bitwise_and(normalized_image, normalized_image, mask=not_mask)

    # Add the bitwise_and results to get the final output image
    output_image = cv2.add(output_image, normalized_not_mask)

    return output_image
  except Exception as e:
    print("Error in flattening image", e)
    return filtered_image
def remove_background(flattened_image):
  try:
    mask = np.zeros(flattened_image.shape[:2], np.uint8)
    rect = (10, 10, flattened_image.shape[1]-20, flattened_image.shape[0]-20)
    cv2.grabCut(flattened_image, mask, rect, None, None, 5, cv2.GC_INIT_WITH_RECT)

    # Create a binary mask from the grabcut mask
    binary_mask = np.where((mask == 2) | (mask == 0), 0, 1).astype(np.uint8)

    # Apply the binary mask to the original image
    flattened_image = cv2.bitwise_and(flattened_image, flattened_image, mask=binary_mask)


    return flattened_image
  except Exception as e:
    print("Error in removing background", e)
    return flattened_image

def get_edge(image):
  try:
    # Apply Scharr operator for x and y directions
    scharr_x = cv2.Scharr(image, cv2.CV_64F, 1, 0)
    scharr_y = cv2.Scharr(image, cv2.CV_64F, 0, 1)

    # Compute the gradient magnitude and direction
    grad_mag = cv2.magnitude(scharr_x, scharr_y)
    grad_dir = cv2.phase(scharr_x, scharr_y, angleInDegrees=True)

    threshold_value = 200  # Adjust this threshold value as needed
    _, edges = cv2.threshold(grad_mag, threshold_value, 255, cv2.THRESH_BINARY)


    edges = edges.astype(np.uint8)
    return edges
  except Exception as e:
    print("Error in getting edge", e)
    return image

def get_contours(edges,image):
  try:
    edge = cv2.Canny(edges, 50, 100)


    # Perform dilation on the edges
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    dilated_edges = cv2.dilate(edge, kernel, iterations=1)

    # Perform erosion on the edges
    eroded_edges = cv2.erode(edge, kernel, iterations=3)
    # Find contours in the binary image
    contours, hierarchy = cv2.findContours(dilated_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnt =max(contours, key=cv2.contourArea)
    cv2.drawContours(image, [cnt], 0, (255, 0, 0), 3)  # green color, line thickness = 3
    
    cv2.drawContours(image, cnt, -1, (0, 255, 0), 2)
    show(image)
    return cnt
  except Exception as e:
    print("Error in getting contours", e)
    return None
def get_features(cnt):
  try:
    moments = cv2.moments(cnt)

    # Calculate circularity
    perimeter = cv2.arcLength(cnt, True)
    area = cv2.contourArea(cnt)
    circularity = 4 * np.pi * area / (perimeter ** 2)
        
    # Calculate rectangularity
    x,y,w,h = cv2.boundingRect(cnt)
    rectangularity = w/h
        
    # Calculate ellipticity
    a = moments['mu20'] + moments['mu02']
    b = np.sqrt(4 * moments['mu11'] ** 2 + (moments['mu20'] - moments['mu02']) ** 2)
    ellipticity = np.sqrt((a + b) / (a - b))

    x, y, w, h = cv2.boundingRect(cnt)
    length = max(w, h)
    width = min(w, h)
    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt, True)
    diameter = getDiameter(cnt, circularity, rectangularity, ellipticity,area)

    apect_ratio = length/width
    form_factor = (4*math.pi*area)/perimeter**2
    narrow_factor = diameter/length
    perimeter_ratio_of_diameter = perimeter/diameter
    perimeter_ratio_of_PLW = perimeter/(length+width)

    # Calculate Eccentricity
    ellipse = cv2.fitEllipse(cnt)
    eccentricity = 1 - (ellipse[1][0] / ellipse[1][1])

    # Calculate Curvature
    epsilon = 0.01 * cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, epsilon, True)
    curvature = len(approx) / cv2.arcLength(cnt, True)

    print('length: '
    )
    features =np.array([length,width,diameter,area,perimeter,rectangularity,circularity,ellipticity,
                        apect_ratio,form_factor,narrow_factor,perimeter_ratio_of_diameter,perimeter_ratio_of_PLW,fractal_dim,
                        entropy, eccentricity,curvature,vein_density, color,species])
    return features

  except Exception as e:
    print("Error in getting features", e)
    return None

def process_image(image):
  # Main
  # Load an image
  try:
    image=resizeImage(image)
    filtered_image = applyFilters(image)
    flattened_image = flatten(filtered_image)
    background_removed = remove_background(flattened_image)
    edge = get_edge(background_removed)
    cnt = get_contours(edge,image)
    features = get_features(cnt)
    return features
  except Exception as e:
    print("Error")

FUNCTIONS FOR INPUTTING DATA TO TABLE


In [ ]:


def append_data_to_csv(file_path, data):
    try:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Create a new DataFrame with the data to be appended
        new_data = pd.DataFrame([data], columns=df.columns)
        
        # Append the new data to the original DataFrame
        df = pd.concat([df, new_data], ignore_index=True)
        
        # Write the updated data back to the CSV file
        df.to_csv(file_path, index=False)
        
        print("Data has been appended to the CSV file successfully!")
    except FileNotFoundError:
        print("File not found. Please check the file path.")
    except Exception as e:
        print("Error occurred:", str(e))


def read_csv_to_table(file_path):
    try:
        # Read CSV file into a pandas DataFrame
        df = pd.read_csv(file_path)      
        return df

    except FileNotFoundError:
        print("File not found. Please check the file path.")
    except Exception as e:
        print("Error occurred:", str(e))

def loop_through_directory(dir_path):
    # Loop through all the contents of the directory
  for item in os.listdir(dir_path):
      # Create the absolute path by joining the directory path and the item name
      item_path = os.path.join(dir_path, item)
      
      # Check if the item is a file
      if os.path.isfile(item_path):
        image_path = item_path

        # Open the image
        image = cv2.imread(image_path)
        data = np.array(process_image(image))
        #append data to csv file
        append_data_to_csv(file_path,data)

**FUNCTION FOR MAIN**

In [ ]:
def main():
    # Specify the directory path
  dir_path = 'images'
  global species
  species = 1
  loop_through_directory(dir_path)
  read_csv_to_table(file_path)

In [ ]:
main()